In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
DATA_IN_PATH = './data/'
DATA_OUT_PATH = './output/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [5]:
sentences = []
for review in reviews:
    sentences.append(review.split())
# word2vec을 사용하기 위해서는 입력 값을 단어로 구분된 리스트로 만들어야 한다.

# Word2Vec

In [9]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
   level=logging.INFO)
# word2vec 학습 과정에서 로그 메세지를 양식에 맞게 INFO 수준으로 보여주는 코드

In [8]:
# word2vec 학습 시 필요한 하이퍼 파라미터
num_features = 300 # 각 단어에 대해 임베딩된 벡터의 차원을 정한다
min_word_count = 40 # 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다.
num_workers = 4 # 모델 학습 시 학습을 위한 프로세스 개수를 지정
context = 10 # 컨텍스트 윈도우 크기
downsampling = 1e-3 # 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다움샘플링 비율을 지정, 보통 0.001이 좋은 성능

In [12]:
from gensim.models import word2vec # !pip install gensim

model = word2vec.Word2Vec(sentences, workers=num_workers,
           size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)

2019-06-27 21:52:22,840 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-27 21:52:22,861 : INFO : collecting all words and their counts
2019-06-27 21:52:22,861 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-27 21:52:23,194 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2019-06-27 21:52:23,473 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2019-06-27 21:52:23,608 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2019-06-27 21:52:23,608 : INFO : Loading a fresh vocabulary
2019-06-27 21:52:23,847 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2019-06-27 21:52:23,847 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2019-06-27 21:52:23,882 : INFO : deleting the raw counts dictionary of 74065 items


In [13]:
model_name = '300features_40minwords_10context'
model.save(model_name)
# 모델을 저장하는 코드, Word2Vec.load()를 통해 다시 사용 가능

2019-06-27 21:55:41,573 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-06-27 21:55:41,573 : INFO : not storing attribute vectors_norm
2019-06-27 21:55:41,589 : INFO : not storing attribute cum_table
c:\python\python36\venv\kmu\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-06-27 21:55:41,875 : INFO : saved 300features_40minwords_10context


# LogisticRegression
1. 학습을 하기 위해서 하나의 리뷰를 같은 형태의 입력값으로 만들어야 한다
2. 위에서 처리한 데이터는 각 단어가 벡터로 표현되어 있다.
3. 리뷰마다 단어의 개수가 다르기 때문에 입력값을 하나의 형태로 만들어야 한다.
4. 가장 단순한 방법으로 문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만든다.

In [14]:
# 하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32) # 출력 벡터 초기화

    num_words = 0
    index2word_set = set(model.wv.index2word)
    # 어휘 사전 준비
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함

    feature_vector = np.divide(feature_vector, num_words)
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로
    return feature_vector

# get_features(words: 리뷰, model: 학습한 word2vec모델, num_features: word2vec으로 임베팅할 때 정한 벡터의 차원 수)
# 하나의 벡터를 만드는 과정에서 속도를 빠르게 하기 위해 np.zeros를 사용해 0만 가지는 벡터를 만든다
# 문장의 단어가 해당 모델 단어사전에 속하는지 보기 위해 model.wv.index2word를 set 객체로 생성
# for문에서 반복문을 통해 리뷰를 구성하는 단어에 대해 임베딩된 벡터가 있는 단어 벡터의 합을 구한다.
# 마지막으로 사용한 단어의 전체 개수로 나눔으로 평균 벡터의 값을 구한다.

In [15]:
# 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

# get_dataset(reviews: 학습 데이터, model: word2vec 모델, num_features: word2vec으로 임베딩할 때 정한 벡터의 차원 수)

In [20]:
test_data_vecs = get_dataset(sentences, model, num_features)

c:\python\python36\venv\kmu\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [25]:
test_data_vecs.shape # np.stack를 사용하여 2차원의 배열을 만듬

(25000, 300)

In [26]:
test_data_vecs

array([[-0.05427849, -0.30749243,  0.04156079, ..., -0.06850925,
         0.12413732,  0.05045112],
       [-0.15822639, -0.19824882,  0.03656638, ..., -0.28203326,
        -0.08932111,  0.08399303],
       [-0.02151882, -0.15708731,  0.05830048, ...,  0.00422297,
         0.11921615, -0.05602044],
       ...,
       [-0.0120516 , -0.14159724, -0.0530271 , ..., -0.02531817,
         0.11852324,  0.27780178],
       [-0.07804546, -0.30542907, -0.00167563, ..., -0.18056087,
         0.11587103,  0.15989284],
       [-0.04820551, -0.4560491 , -0.13065109, ...,  0.0555407 ,
         0.09740985,  0.19358613]], dtype=float32)

## 학습, 검증 데이터셋 분리

In [32]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

## 모델

In [33]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

c:\python\python36\venv\kmu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [34]:
predicted = lgs.predict(X_eval)

In [35]:
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_eval)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("------------")
print("Accuracy: %f" % lgs.score(X_eval, y_eval))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_eval, predicted))
print("Recall: %f" % metrics.recall_score(y_eval, predicted))
print("F1-Score: %f" % metrics.f1_score(y_eval, predicted))
print("AUC: %f" % auc)

------------
Accuracy: 0.865600
Precision: 0.860883
Recall: 0.874553
F1-Score: 0.867664
AUC: 0.934759


## 테스트 데이터 예측

In [36]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [37]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())
# 하나의 문자열을 각 단어의 리스트로 만들기.

In [38]:
test_data_vecs = get_dataset(test_sentences, model, num_features)
# 학습 데이터를 학습시킨 모델을 사용하여 각 리뷰에 대한 특징값을 만든다. (model)

c:\python\python36\venv\kmu\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [39]:
test_predicted = lgs.predict(test_data_vecs)

In [40]:
ids = list(test_data['id'])

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [42]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)

### 캐글에 데이터 제출시 결과 점수 : 0.85920
### public 점수로 비교해본 결과 295등 정도(private 점수는 공개되지 않아 비교 불가)
### TF-IDF를 사용했을 때(0.85368, 314등 정도)보다 약간 오름